In [2]:
import sys, os
sys.version, os.getcwd()

('3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]',
 '/home/jared/galvanize/Projects/prouster')

# Torch

In [50]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/torch-demo/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['French', 'Chinese', 'Dutch', 'Czech', 'Portuguese', 'Russian', 'Italian', 'Spanish', 'Irish', 'Vietnamese', 'Greek', 'Polish', 'Japanese', 'English', 'Scottish', 'Arabic', 'German', 'Korean']
O'Neal


In [51]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [52]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [53]:
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [54]:
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [55]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [56]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [66]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 500
plot_every = n_iters
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 3s (500 0%) 3.8375
0m 5s (1000 1%) 3.4376
0m 8s (1500 1%) 3.8089
0m 11s (2000 2%) 3.2492
0m 14s (2500 2%) 2.9138
0m 17s (3000 3%) 3.5077
0m 20s (3500 3%) 3.0670
0m 23s (4000 4%) 2.7751
0m 26s (4500 4%) 3.0838
0m 30s (5000 5%) 2.4271
0m 33s (5500 5%) 3.3270
0m 36s (6000 6%) 2.8637
0m 39s (6500 6%) 2.5478
0m 41s (7000 7%) 3.0155
0m 44s (7500 7%) 2.3647
0m 47s (8000 8%) 2.9685
0m 50s (8500 8%) 3.7140
0m 53s (9000 9%) 2.5076
0m 56s (9500 9%) 2.0441
1m 0s (10000 10%) 2.7480
1m 2s (10500 10%) 3.1334
1m 5s (11000 11%) 2.5099
1m 8s (11500 11%) 2.6107
1m 10s (12000 12%) 2.8107
1m 13s (12500 12%) 2.8401
1m 16s (13000 13%) 2.7853
1m 19s (13500 13%) 2.8921
1m 21s (14000 14%) 2.3880
1m 24s (14500 14%) 2.5759
1m 26s (15000 15%) 2.8394
1m 29s (15500 15%) 2.0841
1m 32s (16000 16%) 2.7144
1m 35s (16500 16%) 3.2359
1m 38s (17000 17%) 2.2015
1m 41s (17500 17%) 2.6492
1m 44s (18000 18%) 2.1794
1m 46s (18500 18%) 3.0697
1m 49s (19000 19%) 2.2921
1m 52s (19500 19%) 2.9639
1m 55s (20000 20%) 2.4994
1m 57s

In [ ]:
max_length = 50

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(random.randint(1,2))
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUSE')

samples('German', 'GERM')

samples('Spanish', 'SPAN')

samples('Chinese', 'CHIN')

samples('Irish', 'IRLD')

samples('Scottish', 'SCOT')

Rakov
Uanon
Shavan
Eakan
Garter
Eerter
Roun
Merter
Sanala
Parer
Alara
Nara
Cha
Houn
Iun
Nan
Iana
Rongan
Langan
Dangara
Santer
Chang
Oringel
Trang
